# Extract Dataset 

In [2]:
import pandas as pd

In [5]:


tsv_file_path = "./dataset/TikTok_finetuning/TiktokDance"
# Load the TSV file
df = pd.read_csv(f'{tsv_file_path}/train_images.tsv', sep='\t', header=None)


In [3]:
caption_tsv_file_path = "./dataset/TikTok_finetuning/TiktokDance"
# Load the TSV file
df_caption = pd.read_csv(f'{caption_tsv_file_path}/train_captions.tsv', sep='\t', header=None)


In [5]:
# To display full content in all columns and rows
pd.set_option('display.max_colwidth', None)
df_caption.head(10)

,0,1
0,TiktokDance_00001_0001.png,"[{""num_region"": 5, ""image_key"": ""TiktokDance_00001_0001.png"", ""caption"": ""a woman with an orange shirt with butterflies on her shirt."", ""caption_conf"": 0.9147591477022814, ""regions"": [{""box_id"": 0, ""class_name"": ""person"", ""norm_bbox"": [0.4719925820827484, 0.4966033399105072, 0.9439851641654968, 0.9932066798210144], ""conf"": 0.92365962266922, ""region_caption"": ""a woman with an orange shirt with butterflies on her shirt."", ""caption_conf"": 0.9147591477022814}, {""box_id"": 1, ""class_name"": ""butterfly"", ""norm_bbox"": [0.23585689067840576, 0.5092629790306091, 0.14663028717041016, 0.042307592928409576], ""conf"": 0.8738632202148438, ""region_caption"": ""a butterfly sitting on an orange background."", ""caption_conf"": 0.9570571074171874}, {""box_id"": 2, ""class_name"": ""butterfly"", ""norm_bbox"": [0.6667538285255432, 0.5139320492744446, 0.11595204472541809, 0.053857140243053436], ""conf"": 0.7664626836776733, ""region_caption"": ""a brown butterfly sitting on an orange table."", ""caption_conf"": 0.9075698045145534}, {""box_id"": 3, ""class_name"": ""blouse"", ""norm_bbox"": [0.4722321927547455, 0.6591795682907104, 0.9375559091567993, 0.6396914720535278], ""conf"": 0.7001773715019226, ""region_caption"": ""a woman wearing an orange shirt with butterflies on it."", ""caption_conf"": 0.9970340671319324}, {""box_id"": 4, ""class_name"": ""short_pants"", ""norm_bbox"": [0.4413203001022339, 0.8674929141998291, 0.8806557655334473, 0.2626570463180542], ""conf"": 0.6571708917617798, ""region_caption"": ""a person wearing an orange shirt and grey sweatpants."", ""caption_conf"": 0.9430722237121495}]}]"
1,TiktokDance_00001_0002.png,"[{""num_region"": 5, ""image_key"": ""TiktokDance_00001_0002.png"", ""caption"": ""a woman with an orange dress with butterflies on her shirt."", ""caption_conf"": 0.9404542168542169, ""regions"": [{""box_id"": 0, ""class_name"": ""person"", ""norm_bbox"": [0.46692365407943726, 0.4977584183216095, 0.9338473081588745, 0.995516836643219], ""conf"": 0.912740170955658, ""region_caption"": ""a woman with an orange dress with butterflies on her shirt."", ""caption_conf"": 0.9404542168542169}, {""box_id"": 1, ""class_name"": ""butterfly"", ""norm_bbox"": [0.2368704378604889, 0.5088028907775879, 0.1444256454706192, 0.04199704900383949], ""conf"": 0.8738771677017212, ""region_caption"": ""a brown butterfly sitting on an orange background."", ""caption_conf"": 0.9297735554473283}, {""box_id"": 2, ""class_name"": ""butterfly"", ""norm_bbox"": [0.6688584089279175, 0.5137135982513428, 0.11311062425374985, 0.05455022677779198], ""conf"": 0.8287128806114197, ""region_caption"": ""a brown butterfly sitting on an orange wall."", ""caption_conf"": 0.9264783379302365}, {""box_id"": 3, ""class_name"": ""blouse"", ""norm_bbox"": [0.4692786931991577, 0.6465241312980652, 0.9283269643783569, 0.6027728319168091], ""conf"": 0.6851752400398254, ""region_caption"": ""a woman wearing an orange shirt with butterflies on it."", ""caption_conf"": 0.9978814544264754}, {""box_id"": 4, ""class_name"": ""short_pants"", ""norm_bbox"": [0.44008955359458923, 0.8769687414169312, 0.8799525499343872, 0.2431662678718567], ""conf"": 0.6741859316825867, ""region_caption"": ""a person wearing an orange shirt and grey sweatpants."", ""caption_conf"": 0.9731313580907464}]}]"
2,TiktokDance_00001_0003.png,"[{""num_region"": 5, ""image_key"": ""TiktokDance_00001_0003.png"", ""caption"": ""a woman with an orange dress and butterflies on her shirt."", ""caption_conf"": 0.9080405292684351, ""regions"": [{""box_id"": 0, ""class_name"": ""butterfly"", ""norm_bbox"": [0.2398572713136673, 0.5090929865837097, 0.14323873817920685, 0.04179111123085022], ""conf"": 0.918130099773407, ""region_caption"": ""a butterfly sitting on an orange background."", ""caption_conf"": 0.91981609141501}, {""box_id"": 1, ""class_name"": ""person"", ""norm_bbox"": [0.45898568630218506, 0.49936574697494507, 0.9179713726043

In [4]:
import csv
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import base64
from io import BytesIO
import sys

# Increase the CSV field size limit
csv.field_size_limit(sys.maxsize)

# Function to decode base64 image and return a PIL Image
def decode_base64_image(base64_string):
    image_data = base64.b64decode(base64_string)
    image = Image.open(BytesIO(image_data))
    return image

# Function to generate caption using BLIP
def generate_caption(image):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    # Preprocess the image
    inputs = processor(images=image, return_tensors="pt")

    # Check if a GPU is available and move the model to GPU if possible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate the caption
    with torch.no_grad():
        outputs = model.generate(**inputs)

    # Decode the generated caption
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    return caption

# Function to process the TSV and generate captions
def process_tsv(input_tsv, output_tsv):
    with open(input_tsv, 'r') as infile, open(output_tsv, 'w', newline='') as outfile:
        reader = csv.reader(infile, delimiter='\t')
        writer = csv.writer(outfile, delimiter='\t')
        
        # Iterate through each row in the input TSV
        for row in reader:
            image_name = row[0]
            base64_image_string = row[1]
            
            # Decode the image from base64 and generate caption
            image = decode_base64_image(base64_image_string)
            caption = generate_caption(image)
            caption = f"[{{'caption': '{caption}'}}]"
            
            # Write the image name and caption to the output TSV
            writer.writerow([image_name, caption])
            print(f"Processed {image_name}: {caption}")

# Example usage:
input_tsv = "tsv_example/toy_dataset/tsv/train_images.tsv"   # Input TSV containing image names and base64-encoded images
output_tsv = "tsv_example/toy_dataset/tsv/train_captions.tsv"  # Output TSV to save image names and captions

process_tsv(input_tsv, output_tsv)


Processed Custom_000_000001.png: [{'caption': 'a woman with red hair and a gray sweater'}]
Processed Custom_000_000002.png: [{'caption': 'a woman with red hair and a grey sweater'}]
Processed Custom_000_000003.png: [{'caption': 'a woman with red hair and a grey sweater'}]
Processed Custom_000_000004.png: [{'caption': 'a woman with red hair and a grey sweater'}]
Processed Custom_000_000005.png: [{'caption': 'a woman with red hair and a grey sweater'}]
Processed Custom_000_000006.png: [{'caption': 'a woman with red hair and a gray sweater'}]
Processed Custom_000_000007.png: [{'caption': 'a woman with red hair and a gray sweater'}]
Processed Custom_000_000008.png: [{'caption': 'a woman with red hair and a gray sweater'}]
Processed Custom_000_000009.png: [{'caption': 'a woman with red hair and a gray sweater'}]
Processed Custom_001_000001.png: [{'caption': 'a woman in a gray bikini top and a white skirt'}]
Processed Custom_001_000002.png: [{'caption': 'a woman in a gray bikini top and a wh

In [2]:
def generate_lineidx_file(tsv_file, lineidx_file):
    with open(tsv_file, 'r', encoding='utf-8') as f, open(lineidx_file, 'w') as idx_file:
        byte_offset = 0
        for line in f:
            idx_file.write(f"{byte_offset}\n")
            byte_offset += len(line.encode('utf-8'))

# Usage
generate_lineidx_file('tsv_example/toy_dataset/tsv/train_images.tsv', 'tsv_example/toy_dataset/tsv/train_images.lineidx')


In [3]:
import struct

def generate_lineidx_8b(tsv_file, lineidx_8b_file):
    with open(tsv_file, 'r') as tsv_f, open(lineidx_8b_file, 'wb') as lineidx_f:
        pos = 0
        for line in tsv_f:
            # Write the current byte position as a 8-byte (64-bit) integer to the .lineidx.8b file
            lineidx_f.write(struct.pack('q', pos))
            # Move the position forward by the length of the line
            pos += len(line)

# Example usage for captions.tsv
generate_lineidx_8b('tsv_example/toy_dataset/tsv/train_captions.tsv', 'tsv_example/toy_dataset/tsv/train_captions.lineidx.8b')


In [5]:
import csv

def generate_linelist_from_captions(captions_file, output_file):
    """
    Generate a linelist TSV file with indices from a captions TSV file.

    :param captions_file: Path to the input captions TSV file.
    :param output_file: Path to the output linelist TSV file.
    """
    img_line_list = []
    box_line_list = []

    with open(captions_file, 'r') as file:
        reader = csv.reader(file, delimiter='\t')
        for idx, row in enumerate(reader):
            # Assuming row[0] is the image identifier and idx is the index
            img_line_list.append(idx)
            # Use idx for the box index as a placeholder (or modify as needed)
            box_line_list.append(idx)

    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file, delimiter='\t')
        for img_idx, box_idx in zip(img_line_list, box_line_list):
            writer.writerow([img_idx, box_idx])

# Example usage:
captions_path = 'tsv_example/toy_dataset/tsv/train_captions.tsv'
linelist_path = 'tsv_example/toy_dataset/tsv/train_captions.linelist.tsv'
generate_linelist_from_captions(captions_path, linelist_path)


In [19]:
df.head(10)

,0,1
0,TiktokDance_00001_0001.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,TiktokDance_00001_0002.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,TiktokDance_00001_0003.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,TiktokDance_00001_0004.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,TiktokDance_00001_0005.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
5,TiktokDance_00001_0006.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
6,TiktokDance_00001_0007.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
7,TiktokDance_00001_0008.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
8,TiktokDance_00001_0009.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
9,TiktokDance_00001_0010.png,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [17]:
# Filter rows where the first column contains '_00001_' to '_00010_'
filtered_files = df[df.iloc[:, 0].str.contains(r'_0000[1-9]_') | df.iloc[:, 0].str.contains(r'_00015_')]

# Get the count
file_count = len(filtered_files)
file_count

2071

In [18]:
# Print the list of filtered file names
file_list = filtered_files.iloc[:, 0].tolist()
print(file_list)

['TiktokDance_00001_0001.png', 'TiktokDance_00001_0002.png', 'TiktokDance_00001_0003.png', 'TiktokDance_00001_0004.png', 'TiktokDance_00001_0005.png', 'TiktokDance_00001_0006.png', 'TiktokDance_00001_0007.png', 'TiktokDance_00001_0008.png', 'TiktokDance_00001_0009.png', 'TiktokDance_00001_0010.png', 'TiktokDance_00001_0011.png', 'TiktokDance_00001_0012.png', 'TiktokDance_00001_0013.png', 'TiktokDance_00001_0014.png', 'TiktokDance_00001_0015.png', 'TiktokDance_00001_0016.png', 'TiktokDance_00001_0017.png', 'TiktokDance_00001_0018.png', 'TiktokDance_00001_0019.png', 'TiktokDance_00001_0020.png', 'TiktokDance_00001_0021.png', 'TiktokDance_00001_0022.png', 'TiktokDance_00001_0023.png', 'TiktokDance_00001_0024.png', 'TiktokDance_00001_0025.png', 'TiktokDance_00001_0026.png', 'TiktokDance_00001_0027.png', 'TiktokDance_00001_0028.png', 'TiktokDance_00001_0029.png', 'TiktokDance_00001_0030.png', 'TiktokDance_00001_0031.png', 'TiktokDance_00001_0032.png', 'TiktokDance_00001_0033.png', 'TiktokDa

In [ ]:
import os
import shutil

# Define your source directory where the files are located
source_dir = '/path/to/source/directory'

# Define the target base directory
target_base_dir = '/path/to/Target Dataset'

# Function to extract folder number and file number
def extract_numbers(filename):
    parts = filename.split('_')
    folder_number = parts[1]
    file_number = parts[2].split('.')[0]
    return folder_number, file_number

# Copy files to the new structure
for file_name in file_list:
    # Extract numbers from the filename
    folder_number, file_number = extract_numbers(file_name)

    # Create the target directory if it doesn't exist
    target_dir = os.path.join(target_base_dir, folder_number)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Define source and target file paths
    src_file = os.path.join(source_dir, file_name)
    target_file = os.path.join(target_dir, f"{file_number}.png")

    # Copy the file
    shutil.copy(src_file, target_file)
    print(f"Copied {file_name} to {target_file}")

print("File copying completed.")

In [ ]:
# Define the target root folder
target_root = 'Target_Dataset'  # Adjust this to your desired root folder

# Ensure the target root directory exists
os.makedirs(target_root, exist_ok=True)

# Define the range for folder names
folder_range = set(f'{i:03}' for i in range(1, 11))  # Creates set {'001', '002', ..., '010'}

# Loop over the rows in the dataframe
for idx, row in df.iterrows():
    filename = row[0]  # Assuming the first column contains the filename

    # Extract the required folder name (001, 002, etc.) from the filename
    folder_name = filename.split('_')[1][:3]  # Extracts "00001" -> "001"

    # Proceed only if the folder name is in the specified range
    if folder_name in folder_range:
        # Create the target directory (001, 002, etc.)
        target_dir = os.path.join(target_root, folder_name)
        os.makedirs(target_dir, exist_ok=True)

        # Define the source and destination paths
        source_path = os.path.join('source_directory', filename)  # Replace 'source_directory' with your actual source directory path
        target_path = os.path.join(target_dir, os.path.basename(filename))

        # Copy the file to the target location
        shutil.copy(source_path, target_path)

        print(f"Copied {filename} to {target_path}")

print("File organization complete.")

# Get Top 880 Records

In [10]:
import pandas as pd
import os

# Define the base directory paths
base_input_path = r'dataset/TikTok_finetuning/TiktokDance_Main'
base_output_path = r'dataset/TikTok_finetuning/TiktokDance'

# Define the array with values
values = ['images', 'captions', 'masks', 'poses', 'vid2line_offset']

# Loop through each value in the array
for value in values:
    # Generate the input and output file paths dynamically
    input_file = os.path.join(base_input_path, f'new10val_{value}.tsv')
    output_file = os.path.join(base_output_path, f'new10val_{value}.tsv')
    
    # Check if the input file exists
    if os.path.exists(input_file):
        # Read the TSV file
        df = pd.read_csv(input_file, sep='\t')
        
        # Select the first 779 records
        df_subset = df.head(779)
        
        # Write the subset to a new TSV file
        df_subset.to_csv(output_file, sep='\t', index=False)
        
        print(f"Processed file: {input_file} -> {output_file}")
    else:
        print(f"Input file does not exist: {input_file}")


Processed file: dataset/TikTok_finetuning/TiktokDance_Main\new10val_images.tsv -> dataset/TikTok_finetuning/TiktokDance\new10val_images.tsv
Processed file: dataset/TikTok_finetuning/TiktokDance_Main\new10val_captions.tsv -> dataset/TikTok_finetuning/TiktokDance\new10val_captions.tsv
Processed file: dataset/TikTok_finetuning/TiktokDance_Main\new10val_masks.tsv -> dataset/TikTok_finetuning/TiktokDance\new10val_masks.tsv
Processed file: dataset/TikTok_finetuning/TiktokDance_Main\new10val_poses.tsv -> dataset/TikTok_finetuning/TiktokDance\new10val_poses.tsv
Processed file: dataset/TikTok_finetuning/TiktokDance_Main\new10val_vid2line_offset.tsv -> dataset/TikTok_finetuning/TiktokDance\new10val_vid2line_offset.tsv


In [9]:
import os

# Define the base directory paths
base_input_path = r'dataset/TikTok_finetuning/TiktokDance_Main'
base_output_path = r'dataset/TikTok_finetuning/TiktokDance'

# Define the array with values
values = ['images', 'captions', 'masks', 'poses', 'vid2line_offset']

# Loop through each value in the array
for value in values:
    # Generate the input and output file paths dynamically
    input_file = os.path.join(base_input_path, f'train_{value}.lineidx')
    output_file = os.path.join(base_output_path, f'train_{value}.lineidx')
    
    # Check if the input file exists
    if os.path.exists(input_file):
        # Open the input and output files
        with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
            # Copy the first 779 lines from the input file to the output file
            for i, line in enumerate(infile):
                if i < 779:
                    outfile.write(line.rstrip('\n') + '\n')  # Remove extra newline and add just one
                else:
                    break
        
        print(f"Processed file: {input_file} -> {output_file}")
    else:
        print(f"Input file does not exist: {input_file}")


Processed file: dataset/TikTok_finetuning/TiktokDance_Main/train_images.lineidx -> dataset/TikTok_finetuning/TiktokDance/train_images.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/train_captions.lineidx -> dataset/TikTok_finetuning/TiktokDance/train_captions.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/train_masks.lineidx -> dataset/TikTok_finetuning/TiktokDance/train_masks.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/train_poses.lineidx -> dataset/TikTok_finetuning/TiktokDance/train_poses.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/train_vid2line_offset.lineidx -> dataset/TikTok_finetuning/TiktokDance/train_vid2line_offset.lineidx


In [11]:
import os

# Define the base directory paths
base_input_path = r'dataset/TikTok_finetuning/TiktokDance_Main'
base_output_path = r'dataset/TikTok_finetuning/TiktokDance'

# Define the array with values
values = ['images', 'captions', 'masks', 'poses', 'vid2line_offset']

# Loop through each value in the array
for value in values:
    # Generate the input and output file paths dynamically
    input_file = os.path.join(base_input_path, f'new10val_{value}.lineidx')
    output_file = os.path.join(base_output_path, f'new10val_{value}.lineidx')
    
    # Check if the input file exists
    if os.path.exists(input_file):
        # Open the input and output files
        with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
            # Copy the first 779 lines from the input file to the output file
            for i, line in enumerate(infile):
                if i < 400:
                    outfile.write(line.rstrip('\n') + '\n')  # Remove extra newline and add just one
                else:
                    break
        
        print(f"Processed file: {input_file} -> {output_file}")
    else:
        print(f"Input file does not exist: {input_file}")


Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_images.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_images.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_captions.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_captions.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_masks.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_masks.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_poses.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_poses.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_vid2line_offset.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_vid2line_offset.lineidx


In [ ]:
import os

# Define the base directory paths
base_input_path = r'dataset/TikTok_finetuning/TiktokDance_Main'
base_output_path = r'dataset/TikTok_finetuning/TiktokDance'

# Define the array with values
values = ['images', 'captions', 'masks', 'poses', 'vid2line_offset']

# Loop through each value in the array
for value in values:
    # Generate the input and output file paths dynamically
    input_file = os.path.join(base_input_path, f'new10val_{value}.tsv')
    output_file = os.path.join(base_output_path, f'new10val_{value}.tsv')
    
    # Check if the input file exists
    if os.path.exists(input_file):
        # Open the input and output files
        with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
            # Copy the first 779 lines from the input file to the output file
            for i, line in enumerate(infile):
                if i < 400:
                    outfile.write(line.rstrip('\n') + '\n')  # Remove extra newline and add just one
                else:
                    break
        
        print(f"Processed file: {input_file} -> {output_file}")
    else:
        print(f"Input file does not exist: {input_file}")


In [4]:
import pandas as pd
import os

# Define the base directory paths
base_input_path = r'dataset/TikTok_finetuning/TiktokDance_Main'
base_output_path = r'dataset/TikTok_finetuning/TiktokDance'

# Define the array with values
values = ['images', 'captions', 'masks', 'poses', 'vid2line_offset']

# Function to clean the column's string format
def clean_format(value):
    # Replace any instances of escaped quotes with standard quotes
    if isinstance(value, str):
        value = value.replace('""', '"').replace('[{""', '[{"').replace('""}]', '"}]')
    return value

# Loop through each value in the array
for value in values:
    # Generate the input and output file paths dynamically
    input_file = os.path.join(base_input_path, f'new10val_{value}.lineidx')
    output_file = os.path.join(base_output_path, f'new10val_{value}.lineidx')
    
    # Check if the input file exists
    if os.path.exists(input_file):
        # Read the TSV file
        df = pd.read_csv(input_file, sep='\t')
        
        # Clean the format of the 'captions' column if it exists
        if 'captions' in df.columns:
            df['captions'] = df['captions'].apply(clean_format)
        
        # Select the first 400 records
        df_subset = df.head(400)
        
        # Write the cleaned subset to a new TSV file
        df_subset.to_csv(output_file, sep='\t', index=False)
        
        print(f"Processed file: {input_file} -> {output_file}")
    else:
        print(f"Input file does not exist: {input_file}")


Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_images.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_images.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_captions.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_captions.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_masks.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_masks.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_poses.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_poses.lineidx
Processed file: dataset/TikTok_finetuning/TiktokDance_Main/new10val_vid2line_offset.lineidx -> dataset/TikTok_finetuning/TiktokDance/new10val_vid2line_offset.lineidx


779 for training, 400 for validation

In [3]:
import os
import struct

def convert_lineidx_to_8b(lineidx_path, output_path):
    with open(lineidx_path, 'r') as f_in, open(output_path, 'wb') as f_out:
        for line in f_in:
            # Convert each line (offset) to an 8-byte integer (little-endian)
            offset = int(line.strip())
            f_out.write(struct.pack('<Q', offset))

def convert_all_lineidx_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.lineidx'):
            lineidx_path = os.path.join(folder_path, filename)
            output_path = os.path.join(folder_path, filename + '.8b')
            
            print(f"Converting {lineidx_path} to {output_path}...")
            convert_lineidx_to_8b(lineidx_path, output_path)
            print(f"Conversion completed: {output_path}")

# Example usage
folder_path = base_output_path = r'dataset/TikTok_finetuning/TiktokDance'  # Replace with your folder path
#folder_path = base_output_path = r'dataset/TikTok_finetuning/composite_offset'  # Replace with your folder path
convert_all_lineidx_in_folder(folder_path)


Converting dataset/TikTok_finetuning/TiktokDance/new10val_poses.lineidx to dataset/TikTok_finetuning/TiktokDance/new10val_poses.lineidx.8b...
Conversion completed: dataset/TikTok_finetuning/TiktokDance/new10val_poses.lineidx.8b
Converting dataset/TikTok_finetuning/TiktokDance/train_poses.lineidx to dataset/TikTok_finetuning/TiktokDance/train_poses.lineidx.8b...
Conversion completed: dataset/TikTok_finetuning/TiktokDance/train_poses.lineidx.8b
Converting dataset/TikTok_finetuning/TiktokDance/new10val_vid2line_offset.lineidx to dataset/TikTok_finetuning/TiktokDance/new10val_vid2line_offset.lineidx.8b...
Conversion completed: dataset/TikTok_finetuning/TiktokDance/new10val_vid2line_offset.lineidx.8b
Converting dataset/TikTok_finetuning/TiktokDance/new10val_masks.lineidx to dataset/TikTok_finetuning/TiktokDance/new10val_masks.lineidx.8b...
Conversion completed: dataset/TikTok_finetuning/TiktokDance/new10val_masks.lineidx.8b
Converting dataset/TikTok_finetuning/TiktokDance/new10val_captions.